In [1]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.models import ResNet18_Weights.DEFAULT
from torchvision.datasets import ImageFolder
from torcheval.metrics.functional import binary_confusion_matrix 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def create_dataloader(dataset:str,
                      batch_size: int=16,
                      pin_memory: bool=True,
                      shuffle: bool=True) -> torch.utils.data.DataLoader:

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=pin_memory
    )    
    
    return loader   

def train_step(model: nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: nn.Module,
              optimizer: torch.optim.Optimizer,
              device: str,
              validation: bool=False) -> float:
    loss = 0
    if not validation:
        
        model.train()
        
        for X, y in loader:
            
            X, y = X.to(device), y.to(device)
    
            logits = model(X)#.squeeze()
            preds = logits.sigmoid().round()
    
            batch_loss = loss_fn(logits, y)
            loss += batch_loss.item()
            cm = binary_comfusion_matrix(preds, y)
            
            optimizer.zero_grad()
            batch_loss.backward()
            oprimizer.step()
        
    if validation:
        
        model.eval()
        with torch.inference_mode():

            for X, y in loader:
                
            X, y = X.to(device), y.to(device)

            logits = model(X)#.squeeze()
            preds = logints.sigmoid().round()

            batch_loss = loss_fn(logits, y)
            loss += batch_loss
            cm = binary_comfusion_matrix(preds, y)
            
    return loss

In [ ]:
# download model and weights
weights = ResNet17_Weights.DEFAULT
transforms_auto = weights.transforms()

In [ ]:
path_train = '../mammo/png_roi/train_images'
path_test = '../mammo/png_roi/test_images'

dataset_train = ImageFolder(path_train, transform=transforms_auto)
dataset_test = ImageFolder(path_test, transform=transforms_auto)

loader_train = create_dataloader(dataset_train)
loader_test = create_dataloader(dataset_test, shuffle=False)

In [ ]:
model = resnet18(weights=weights).to(device)
lr = 3e-4
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
n_epochs = 1

for epoch in tqdm(range(n_epochs)):

    print(f'Epoch #{epoch+1}')
    
    train_loss = 0
    test_loss = 0
    train_loss += train_step(model, loader_train, loss_fn, optimizer, device)
    test_loss += train_step(model, loader_test, loss_fn, optimizer, device)

    if epoch % 5 == 0:
        print(f'Train loss: [{loss_train}] Test loss: [{loss_test}]')

In [2]:
a = 0
b = 0
def ass(a):
    return a*2, a*3
a, b += ass(2)
a, b

SyntaxError: 'tuple' is an illegal expression for augmented assignment (1568998929.py, line 5)